In [1]:
import json
import pathlib
from tqdm import tqdm

In [2]:
# Construct single json file
data_dir = pathlib.Path('./data/')
json_paths = sorted(data_dir.glob('*.json'))
out_json = pathlib.Path('./merged.json')
data = []
for path in tqdm(json_paths):
    with path.open() as f:
        data.append(json.load(f))
with out_json.open('w') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

100%|██████████| 1266/1266 [00:00<00:00, 11875.13it/s]


In [3]:
%%bash
# Start solr
../solr-7.3.0/bin/solr start
# Create core
../solr-7.3.0/bin/solr create -c recipe

*** [WARN] *** Your open file limit is currently 1024.  
 It should be set to 65000 to avoid operational disruption. 
 If you no longer wish to see this warning, set SOLR_ULIMIT_CHECKS to false in your profile or solr.in.sh
*** [WARN] ***  Your Max Processes Limit is currently 31295. 
 It should be set to 65000 to avoid operational disruption. 
 If you no longer wish to see this warning, set SOLR_ULIMIT_CHECKS to false in your profile or solr.in.sh
Waiting up to 180 seconds to see Solr running on port 8983 [|]   [/]   [-]   [\]   [|]   [/]   [-]   [\]   [|]   [/]   [-]   [\]  
Started Solr server on port 8983 (pid=32001). Happy searching!

    WARNING: Using _default configset with data driven schema functionality. NOT RECOMMENDED for production use.
         To turn off: bin/solr config -c recipe -p 8983 -property update.autoCreateFields -value false

Created new core 'recipe'


In [4]:
%%bash
# Commit
curl 'http://localhost:8983/solr/recipe/update?commit=true' \
    --data-binary @./merged.json \
    -H 'Content-type:application/json'

{
  "responseHeader":{
    "status":0,
    "QTime":3152}}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2708k  100    58  100 2708k     18   857k  0:00:03  0:00:03 --:--:--  857k


In [7]:
%%bash

curl http://localhost:8983/solr/recipe/query -d '
{
  "query": "(ingr:lettuce AND ingr:onion AND ingr:tomato AND ingr:pasta) (n_ingr:[2 TO 6]) (n_ingr:2)^0.5 (n_ingr:3)^0.8 (n_ingr:4)^1.5 (n_ingr:5)^0.8 (n_ingr:6)^0.5",
  "limit": 1,
  "offset": 10,
}'

{
  "responseHeader":{
    "status":0,
    "QTime":19,
    "params":{
      "json":"\n{\n  \"query\": \"(ingr:lettuce AND ingr:onion AND ingr:tomato AND ingr:pasta) (n_ingr:[2 TO 6]) (n_ingr:2)^0.5 (n_ingr:3)^0.8 (n_ingr:4)^1.5 (n_ingr:5)^0.8 (n_ingr:6)^0.5\",\n  \"limit\": 1,\n  \"offset\": 10,\n}"}},
  "response":{"numFound":67,"start":10,"docs":[
      {
        "slug":["beetroot-crisps"],
        "title":["Beetroot Crisps Recipe by Tasty"],
        "n_step":[7],
        "n_ingr":[3],
        "ingr":["3 bulbs beetroot",
          "1 l (4 ¼ cups) oil, for frying",
          "salt, to taste"],
        "step":["start by preparing your beetroot. wash thoroughly under water and remove the stems and root. next, peel the beetroot and cut each bulb in half.",
          "next, slice your beetroot as thinly as you can. for ease and speed, it is useful to use a food processor with the slicing attachment; if you don’t have one, just do your best to cut the beetroot as thinly as you can.",
     

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3286  100  3084  100   202  96375   6312 --:--:-- --:--:-- --:--:--  100k


In [70]:
%%bash
# Delete core
curl "http://localhost:8983/solr/admin/cores?action=UNLOAD&core=recipe&deleteIndex=true&deleteDataDir=true&deleteInstanceDir=true"
# Shutdown
../solr-7.3.0/bin/solr stop -all

{
  "responseHeader":{
    "status":0,
    "QTime":20}}
Sending stop command to Solr running on port 8983 ... waiting up to 180 seconds to allow Jetty process 13340 to stop gracefully.
    

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    56  100    56    0     0   1806      0 --:--:-- --:--:-- --:--:--  1806
